## Code for Linear Regression and Decision Tree

In [92]:
import pandas as pd
#pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', None)
#pd.set_option('display.width', 1000)

import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import DecisionTreeRegressor
from datetime import timezone, datetime
from sklearn.preprocessing import OneHotEncoder 
from sklearn.preprocessing import LabelEncoder
#from sklearn.compose import ColumnTransformer 
import datetime as dt
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import KFold 
from sklearn.metrics import mean_squared_error as MSE
from sklearn.metrics import classification_report, confusion_matrix

### Importing Dataset
hingek10.csv and hingek20.csv are the training results of the low rank model of clustering the patients' information. Diabetes_Numerical.csv is the dataset used for training linear regression and decision tree models.

In [93]:
hinge10 = pd.read_csv('../data/hingek10.csv')
#hinge20 = pd.read_csv('../data/hingek20.csv')
origin20 = pd.read_csv('../data/GLRMk20.csv')
origin5 = pd.read_csv('../data/GLRMk5.csv')
df_pat = pd.read_csv('../data/Diabetes_Numerical.csv')

hinge7 = pd.read_csv('../data/hingek7.csv')
param7 = pd.read_csv('../data/hingeparam7.csv')
hinge6 = pd.read_csv('../data/hingek6.csv')
param6 = pd.read_csv('../data/hingeparam6.csv')
hinge4 = pd.read_csv('../data/hingek4.csv')



param100_7=pd.read_csv('../data/hingeparam100.7.csv')
param100_6= pd.read_csv('../data/hingeparam100.6.csv')
param100_4= pd.read_csv('../data/hingeparam100.4.csv')

In [45]:
param100_4

,x1,x2,x3,x4,x5,x6,x7,x8,x9,x10,...,x10301,x10302,x10303,x10304,x10305,x10306,x10307,x10308,x10309,x10310
0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0
2,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0
3,0.0,1.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [46]:
hinge4

,x1,x2,x3,x4,x5,x6,x7,x8,x9,x10,...,x10301,x10302,x10303,x10304,x10305,x10306,x10307,x10308,x10309,x10310
0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,...,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0
2,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,1.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0
3,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0


## Function for regression training

In [94]:
def train_LR(X_train, X_test, y_train, y_test, i):
    # create training and testing vars
    #X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25)
    print(X_train.shape, y_train.shape)
    print(X_test.shape, y_test.shape)
    lm = LinearRegression()
    model = lm.fit(X_train, y_train)
    y_pred = lm.predict(X_test)
    print("Score:", MSE(y_pred, y_test), '\n')
    #print(confusion_matrix(y_test, y_pred))
    #print(classification_report(y_test, y_pred))

In [95]:
def train_DT(X_train, X_test, y_train, y_test, i):
    # create training and testing vars
    print(X_train.shape, y_train.shape)
    print(X_test.shape, y_test.shape)
    dt = DecisionTreeRegressor()
    model = dt.fit(X_train, y_train)
    y_pred = dt.predict(X_test)
    print("Score:", MSE(y_pred, y_test), '\n')
    #print(confusion_matrix(y_test, y_pred))
    #print(classification_report(y_test, y_pred))

In [49]:
def trainModel(df_clus, df_pat, size):
    df_clus = df_clus.transpose()
    
    # get the clusters
    clusters = []
    for i in range(size):
        this_cluster = df_clus[df_clus[i] == 1]
        clusters.append(this_cluster)
    
    # get the index (SubjectID)
    cluster = []
    for i in range(size):
        this_cluster = list(map(lambda x: int(x[1:]) - 1, list(clusters[i].index)))
        cluster.append(this_cluster)
        
    # split the patient dataframe
    pat_clustered = []
    for i in range(size):
        this_clu = df_pat[df_pat.index.isin(cluster[i])]
        pat_clustered.append(this_clu)
    for i in range(size):
        this_c = pat_clustered[i]
        if (len(this_c) > 0):
            this_df = this_c.iloc[:, 1:] #this_c.iloc[:, 2:] for 
            y = this_df.LOS
            X = this_df.iloc[:, 1:]
            X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25)
            print('\n Cluster '+ str(i+1)+ ': \n')
            print('Linear Regression: ')
            train_LR(X_train, X_test, y_train, y_test, i)
            print('Decission Tree: ')
            train_DT(X_train, X_test, y_train, y_test, i)
        else:
            print('Cluster'+ str(i+1)+ ' is empty. \n')

## Model Output

In [50]:
trainModel(hinge4, df_pat, 4)


 Cluster 1: 

Linear Regression: 
(1947, 919) (1947,)
(650, 919) (650,)
Score: 1.5978054899701247e+21 

Decission Tree: 
(1947, 919) (1947,)
(650, 919) (650,)
Score: 71.07275675466538 


 Cluster 2: 

Linear Regression: 
(1937, 919) (1937,)
(646, 919) (646,)
Score: 8.714307290317095e+21 

Decission Tree: 
(1937, 919) (1937,)
(646, 919) (646,)
Score: 126.62211856517415 


 Cluster 3: 

Linear Regression: 
(2028, 919) (2028,)
(676, 919) (676,)
Score: 1.6016118399737103e+23 

Decission Tree: 
(2028, 919) (2028,)
(676, 919) (676,)
Score: 114.20062898677885 


 Cluster 4: 

Linear Regression: 
(1819, 919) (1819,)
(607, 919) (607,)
Score: 6.465389092682245e+19 

Decission Tree: 
(1819, 919) (1819,)
(607, 919) (607,)
Score: 136.53558997669276 



In [51]:
trainModel(param100_4, df_pat, 4)


 Cluster 1: 

Linear Regression: 
(1811, 919) (1811,)
(604, 919) (604,)
Score: 4.9916518171999885e+17 

Decission Tree: 
(1811, 919) (1811,)
(604, 919) (604,)
Score: 95.92851193960264 


 Cluster 2: 

Linear Regression: 
(2017, 919) (2017,)
(673, 919) (673,)
Score: 1.0201576402729765e+22 

Decission Tree: 
(2017, 919) (2017,)
(673, 919) (673,)
Score: 127.72917139157506 


 Cluster 3: 

Linear Regression: 
(1970, 919) (1970,)
(657, 919) (657,)
Score: 5.595175464499356e+22 

Decission Tree: 
(1970, 919) (1970,)
(657, 919) (657,)
Score: 92.4829874758904 


 Cluster 4: 

Linear Regression: 
(1933, 919) (1933,)
(645, 919) (645,)
Score: 8.6894988581799e+22 

Decission Tree: 
(1933, 919) (1933,)
(645, 919) (645,)
Score: 121.85875849432557 



In [10]:
trainModel(hinge6, df_pat, 6)


 Cluster 1: 

Linear Regression: 
(1257, 919) (1257,)
(419, 919) (419,)
Score: 3.216515560609482e+21 

Decission Tree: 
(1257, 919) (1257,)
(419, 919) (419,)
Score: 91.79407462701668 


 Cluster 2: 

Linear Regression: 
(1232, 919) (1232,)
(411, 919) (411,)
Score: 4.025143679903894e+21 

Decission Tree: 
(1232, 919) (1232,)
(411, 919) (411,)
Score: 108.23295410287105 


 Cluster 3: 

Linear Regression: 
(1333, 919) (1333,)
(445, 919) (445,)
Score: 6.59281801768297e+22 

Decission Tree: 
(1333, 919) (1333,)
(445, 919) (445,)
Score: 211.3686790483595 


 Cluster 4: 

Linear Regression: 
(1257, 919) (1257,)
(419, 919) (419,)
Score: 7.042571286663099e+21 

Decission Tree: 
(1257, 919) (1257,)
(419, 919) (419,)
Score: 135.3733607617661 


 Cluster 5: 

Linear Regression: 
(1287, 919) (1287,)
(430, 919) (430,)
Score: 8.668875255000166e+22 

Decission Tree: 
(1287, 919) (1287,)
(430, 919) (430,)
Score: 205.66740605137792 


 Cluster 6: 

Linear Regression: 
(1365, 919) (1365,)
(455, 919) (45

In [11]:
trainModel(param6, df_pat, 6)


 Cluster 1: 

Linear Regression: 
(1371, 919) (1371,)
(458, 919) (458,)
Score: 4.205420198817009e+22 

Decission Tree: 
(1371, 919) (1371,)
(458, 919) (458,)
Score: 114.02648763730895 


 Cluster 2: 

Linear Regression: 
(1236, 919) (1236,)
(412, 919) (412,)
Score: 2.6192324732176546e+23 

Decission Tree: 
(1236, 919) (1236,)
(412, 919) (412,)
Score: 81.28964212633495 


 Cluster 3: 

Linear Regression: 
(1248, 919) (1248,)
(417, 919) (417,)
Score: 2.1302903581164825e+23 

Decission Tree: 
(1248, 919) (1248,)
(417, 919) (417,)
Score: 100.57983526203837 


 Cluster 4: 

Linear Regression: 
(1278, 919) (1278,)
(426, 919) (426,)
Score: 1.00769997961152e+22 

Decission Tree: 
(1278, 919) (1278,)
(426, 919) (426,)
Score: 130.73175677514084 


 Cluster 5: 

Linear Regression: 
(1259, 919) (1259,)
(420, 919) (420,)
Score: 1.5927785606854505e+22 

Decission Tree: 
(1259, 919) (1259,)
(420, 919) (420,)
Score: 168.43410734888693 


 Cluster 6: 

Linear Regression: 
(1338, 919) (1338,)
(447, 919

In [12]:
trainModel(param100_6, df_pat, 6)


 Cluster 1: 

Linear Regression: 
(1218, 919) (1218,)
(406, 919) (406,)
Score: 4.7635150392034456e+23 

Decission Tree: 
(1218, 919) (1218,)
(406, 919) (406,)
Score: 118.60164252778323 


 Cluster 2: 

Linear Regression: 
(1322, 919) (1322,)
(441, 919) (441,)
Score: 6.490827392489847e+22 

Decission Tree: 
(1322, 919) (1322,)
(441, 919) (441,)
Score: 98.09954845387755 


 Cluster 3: 

Linear Regression: 
(1268, 919) (1268,)
(423, 919) (423,)
Score: 4.1813587583229914e+23 

Decission Tree: 
(1268, 919) (1268,)
(423, 919) (423,)
Score: 103.82216194321512 


 Cluster 4: 

Linear Regression: 
(1341, 919) (1341,)
(447, 919) (447,)
Score: 4.274020866003761e+22 

Decission Tree: 
(1341, 919) (1341,)
(447, 919) (447,)
Score: 170.67940930799213 


 Cluster 5: 

Linear Regression: 
(1268, 919) (1268,)
(423, 919) (423,)
Score: 4.9978121206620335e+25 

Decission Tree: 
(1268, 919) (1268,)
(423, 919) (423,)
Score: 76.38225196635935 


 Cluster 6: 

Linear Regression: 
(1314, 919) (1314,)
(439, 919

In [14]:
trainModel(hinge7, df_pat, 7)


 Cluster 1: 

Linear Regression: 
(1145, 919) (1145,)
(382, 919) (382,)
Score: 2.07982022474958e+22 

Decission Tree: 
(1145, 919) (1145,)
(382, 919) (382,)
Score: 101.47224894942408 


 Cluster 2: 

Linear Regression: 
(998, 919) (998,)
(333, 919) (333,)
Score: 1.2304106982108725e+22 

Decission Tree: 
(998, 919) (998,)
(333, 919) (333,)
Score: 77.78637617442192 


 Cluster 3: 

Linear Regression: 
(1014, 919) (1014,)
(338, 919) (338,)
Score: 4.231011223278899e+21 

Decission Tree: 
(1014, 919) (1014,)
(338, 919) (338,)
Score: 126.15630863545118 


 Cluster 4: 

Linear Regression: 
(1127, 919) (1127,)
(376, 919) (376,)
Score: 2.231739565070974e+22 

Decission Tree: 
(1127, 919) (1127,)
(376, 919) (376,)
Score: 89.63891348498007 


 Cluster 5: 

Linear Regression: 
(1102, 919) (1102,)
(368, 919) (368,)
Score: 4.653746301119692e+23 

Decission Tree: 
(1102, 919) (1102,)
(368, 919) (368,)
Score: 114.44080187138589 


 Cluster 6: 

Linear Regression: 
(1203, 919) (1203,)
(401, 919) (401,

In [15]:
trainModel(param7, df_pat, 7)


 Cluster 1: 

Linear Regression: 
(1092, 919) (1092,)
(365, 919) (365,)
Score: 1.671156647680722e+20 

Decission Tree: 
(1092, 919) (1092,)
(365, 919) (365,)
Score: 82.14916316991781 


 Cluster 2: 

Linear Regression: 
(1136, 919) (1136,)
(379, 919) (379,)
Score: 8.806395083422193e+21 

Decission Tree: 
(1136, 919) (1136,)
(379, 919) (379,)
Score: 179.33665872450527 


 Cluster 3: 

Linear Regression: 
(1071, 919) (1071,)
(357, 919) (357,)
Score: 3.016284724429457e+22 

Decission Tree: 
(1071, 919) (1071,)
(357, 919) (357,)
Score: 160.21334614529414 


 Cluster 4: 

Linear Regression: 
(1095, 919) (1095,)
(366, 919) (366,)
Score: 1.4649226022227154e+22 

Decission Tree: 
(1095, 919) (1095,)
(366, 919) (366,)
Score: 106.96927064423498 


 Cluster 5: 

Linear Regression: 
(1131, 919) (1131,)
(378, 919) (378,)
Score: 5.098140831252453e+21 

Decission Tree: 
(1131, 919) (1131,)
(378, 919) (378,)
Score: 132.94926634913358 


 Cluster 6: 

Linear Regression: 
(1097, 919) (1097,)
(366, 919)

In [74]:
trainModel(param100_7, df_pat, 7)


 Cluster 1: 

Linear Regression: 
(1275, 919) (1275,)
(426, 919) (426,)
Score: 9.599924462046514e+22 

Decission Tree: 
(1275, 919) (1275,)
(426, 919) (426,)
Score: 76.20507517715961 


 Cluster 2: 

Linear Regression: 
(1092, 919) (1092,)
(365, 919) (365,)
Score: 1.0311859342926146e+21 

Decission Tree: 
(1092, 919) (1092,)
(365, 919) (365,)
Score: 54.55866417457535 


 Cluster 3: 

Linear Regression: 
(1026, 919) (1026,)
(342, 919) (342,)
Score: 1.802659239418777e+23 

Decission Tree: 
(1026, 919) (1026,)
(342, 919) (342,)
Score: 99.30641366163742 


 Cluster 4: 

Linear Regression: 
(1064, 919) (1064,)
(355, 919) (355,)
Score: 2.4075310357751645e+23 

Decission Tree: 
(1064, 919) (1064,)
(355, 919) (355,)
Score: 121.01817180949297 


 Cluster 5: 

Linear Regression: 
(1155, 919) (1155,)
(386, 919) (386,)
Score: 7.623448887043581e+23 

Decission Tree: 
(1155, 919) (1155,)
(386, 919) (386,)
Score: 147.41978776310881 


 Cluster 6: 

Linear Regression: 
(1101, 919) (1101,)
(368, 919) 

In [13]:
#trainModel(hinge20, df_pat, 20)

In [30]:
trainModel(hinge10, df_pat, 10)


 Cluster 1: 

Linear Regression: 
(728, 919) (728,)
(243, 919) (243,)
Score: 1.547640555943283e+23 

Decission Tree: 
(728, 919) (728,)
(243, 919) (243,)
Score: 123.37350409773661 


 Cluster 2: 

Linear Regression: 
(756, 919) (756,)
(252, 919) (252,)
Score: 4.858488139807545e+22 

Decission Tree: 
(756, 919) (756,)
(252, 919) (252,)
Score: 111.49082908115079 


 Cluster 3: 

Linear Regression: 
(725, 919) (725,)
(242, 919) (242,)
Score: 1.7823168930889417e+23 

Decission Tree: 
(725, 919) (725,)
(242, 919) (242,)
Score: 92.0645657770248 


 Cluster 4: 

Linear Regression: 
(785, 919) (785,)
(262, 919) (262,)
Score: 3.7093980363677898e+22 

Decission Tree: 
(785, 919) (785,)
(262, 919) (262,)
Score: 174.61384428030533 


 Cluster 5: 

Linear Regression: 
(769, 919) (769,)
(257, 919) (257,)
Score: 3.5866619933918457e+22 

Decission Tree: 
(769, 919) (769,)
(257, 919) (257,)
Score: 89.39794778447471 


 Cluster 6: 

Linear Regression: 
(739, 919) (739,)
(247, 919) (247,)
Score: 5.02965

In [31]:
trainModel(origin20, df_pat, 20)


 Cluster 1: 

Linear Regression: 
(2, 919) (2,)
(1, 919) (1,)
Score: 0.6404185299408283 

Decission Tree: 
(2, 919) (2,)
(1, 919) (1,)
Score: 1.3064489999999995 


 Cluster 2: 

Linear Regression: 
(844, 919) (844,)
(282, 919) (282,)
Score: 2.771791993502745e+21 

Decission Tree: 
(844, 919) (844,)
(282, 919) (282,)
Score: 8.233064853829788 


 Cluster 3: 

Linear Regression: 
(2, 919) (2,)
(1, 919) (1,)
Score: 76.62680296963988 

Decission Tree: 
(2, 919) (2,)
(1, 919) (1,)
Score: 137.82760000000002 


 Cluster 4: 

Linear Regression: 
(1214, 919) (1214,)
(405, 919) (405,)
Score: 1.2931379920447186e+21 

Decission Tree: 
(1214, 919) (1214,)
(405, 919) (405,)
Score: 423.1826870244259 


 Cluster 5: 

Linear Regression: 
(76, 919) (76,)
(26, 919) (26,)
Score: 121.13836947259418 

Decission Tree: 
(76, 919) (76,)
(26, 919) (26,)
Score: 98.10329950576923 


 Cluster 6: 

Linear Regression: 
(75, 919) (75,)
(26, 919) (26,)
Score: 25.63677034884739 

Decission Tree: 
(75, 919) (75,)
(26, 9

In [ ]:
trainModel(origin5, df_pat, 5)

# Print Clusters

In [100]:
def printcluster(df_clus, df_pat, size):
    df_clus = df_clus.transpose()
    
    # get the clusters
    clusters = []
    for i in range(size):
        this_cluster = df_clus[df_clus[i] == 1]
        clusters.append(this_cluster)
    #print(clusters)
    # get the index
    cluster = []
    for i in range(size):
        this_cluster = list(map(lambda x: int(x[1:]) - 1, list(clusters[i].index)))
        cluster.append(this_cluster)
        
    # split the patient dataframe
    pat_clustered = []
    for i in range(size):
        this_clu = df_pat[df_pat.index.isin(cluster[i])]
        pat_clustered.append(this_clu)
        
        
    for i in range(size):
        this_c = pat_clustered[i]

        
        
        if (len(this_c) > 0):
            this_df = this_c.iloc[:, 1:]
            y = this_df.LOS
            X = this_df.iloc[:, 1:]
            X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25)
            print('\n Cluster '+ str(i+1)+ ': \n')
            print('Linear Regression: ')
            train_LR(X_train, X_test, y_train, y_test, i)
            print('Decission Tree: ')
            train_DT(X_train, X_test, y_train, y_test, i)
        else:
            print('Cluster'+ str(i+1)+ ' is empty. \n')
    
    #pat_clustered = pd.DataFrame(pat_clustered)
    for i in range(size):
        #print (pat_clustered[i])
        print('\n Cluster' + str(i+1) + 'Summary Statistics \n')
        #a = pat_clustered.iloc[:, i]
        #print(a.describe())
        #print(pat_clustered.iloc[:, i].describe())
        
        
        #print(pat_clustered[i].describe())
        pat_clustered[i].describe()


# PrintClusters2 

In [97]:
size = 4 
df_clus = param100_4

df_clus = df_clus.transpose()

# get the clusters
clusters = []
for i in range(size):
    this_cluster = df_clus[df_clus[i] == 1]
    clusters.append(this_cluster)
#print(clusters)
# get the index
cluster = []
for i in range(size):
    this_cluster = list(map(lambda x: int(x[1:]) - 1, list(clusters[i].index)))
    cluster.append(this_cluster)


# split the patient dataframe
pat_clustered = []
for i in range(size):
    this_clu = df_pat[df_pat.index.isin(cluster[i])]
    pat_clustered.append(this_clu)
    
for i in range(size):
    this_c = pat_clustered[i]




In [102]:

print('\n Cluster' + str(0+1) + '\n Summary Statistics \n')
pat_clustered[0].describe()


 Cluster1
 Summary Statistics 



,SUBJECT_ID,LOS,HOSPITALIZATION,ORDINAL_AGE,ELECTIVE,EMERGENCY,NEWBORN,URGENT,Government,Medicaid,Medicare,Private,Self Pay,DIVORCED,LIFE PARTNER,MARRIED,NAN,SEPARATED,SINGLE,UNKNOWN,WIDOWED,GENDER,3,4,5,7,8,9,10,11,12,13,14,18,27,31,32,34,35,38,39,40,41,42,46,47,48,49,52,53,54,57,58,66,70,75,78,79,82,86,88,94,96,97,110,111,112,117,120,121,127,130,131,135,136,137,138,139,140,141,142,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,161,162,163,164,171,172,173,174,175,176,180,182,183,184,185,186,187,188,189,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,207,208,209,210,211,212,213,214,215,216,217,218,220,221,223,225,226,227,228,229,230,232,233,235,236,237,238,239,240,241,242,244,245,246,249,250,251,252,253,254,255,256,257,258,259,261,262,263,265,266,268,269,270,271,272,273,274,275,276,277,278,279,280,281,282,283,284,285,286,287,288,289,290,291,292,293,294,295,296,297,298,299,300,301,303,304,305,306,307,308,309,310,311,312,313,314,315,317,318,319,320,321,322,323,324,325,327,331,332,333,334,335,336,337,338,339,340,341,342,343,344,345,346,347,348,349,350,351,352,353,354,355,356,357,358,359,360,361,362,363,364,365,366,367,368,369,370,371,372,373,374,375,376,377,378,379,380,382,383,385,386,387,388,389,390,391,394,395,396,397,398,401,402,403,404,405,410,411,412,413,414,415,416,417,420,421,422,423,424,425,426,427,428,429,430,431,432,433,434,435,436,437,438,440,441,442,443,444,445,446,447,448,449,451,452,453,454,455,456,457,458,459,461,462,463,464,465,466,471,472,473,475,477,478,480,481,482,483,484,485,486,487,488,490,491,492,493,494,495,496,500,501,502,506,507,508,510,511,512,513,514,515,516,517,518,519,520,521,522,523,524,525,526,527,528,529,530,531,532,533,534,535,536,537,538,540,541,542,543,550,551,552,553,555,556,557,558,560,562,564,565,566,567,568,569,570,571,572,573,574,575,576,577,578,579,580,581,582,583,584,585,586,587,588,590,591,592,593,594,595,596,597,598,599,600,601,602,603,604,605,607,608,611,614,615,616,617,618,619,620,621,622,623,624,625,626,627,628,629,632,633,634,642,644,646,647,648,649,652,654,666,670,674,680,681,682,683,684,685,686,690,692,693,694,695,696,697,698,700,701,702,703,704,705,706,707,708,709,710,711,712,713,714,715,716,718,719,720,721,722,723,724,725,726,727,728,729,730,731,732,733,734,735,736,737,738,741,742,745,746,747,748,750,751,752,753,754,755,756,757,758,759,765,766,770,774,775,780,781,782,783,784,785,786,787,788,789,790,791,792,793,794,795,796,799,800,801,802,803,804,805,806,807,808,810,811,812,813,814,815,816,817,820,821,822,823,824,825,826,830,831,833,834,835,836,839,840,842,845,846,847,850,851,852,853,854,860,861,862,863,864,865,866,867,868,869,870,872,873,874,875,879,880,881,882,883,884,886,887,890,891,892,893,894,896,897,900,901,902,903,904,905,906,907,908,909,910,911,912,913,914,916,917,918,919,920,921,922,923,924,926,927,933,934,935,936,937,939,941,942,944,945,947,948,950,951,952,953,954,955,956,958,959,961,962,964,965,966,967,968,969,970,971,972,974,975,977,983,987,989,990,991,992,994,995,996,997,998,999,E812,E813,E814,E815,E816,E817,E818,E819,E821,E822,E823,E824,E825,E826,E838,E848,E849,E850,E851,E852,E853,E854,E855,E857,E858,E866,E869,E870,E871,E874,E876,E878,E879,E880,E881,E882,E884,E885,E887,E888,E890,E899,E900,E901,E906,E910,E911,E912,E915,E916,E917,E918,E920,E922,E924,E927,E928,E929,E930,E931,E932,E933,E934,E935,E936,E937,E938,E939,E940,E941,E942,E943,E944,E945,E946,E947,E950,E953,E955,E956,E957,E958,E960,E965,E966,E967,E968,E969,E980,E988,E989,V01,V02,V03,V04,V05,V06,V07,V08,V09,V10,V11,V12,V13,V14,V15,V16,V17,V18,V19,V23,V26,V27,V29,V30,V42,V43,V44,V45,V46,V49,V50,V53,V54,V55,V56,V58,V60,V61,V62,V63,V64,V65,V66,V69,V70,V72,V74,V76,DEAD,ASIAN,WHITE,UNKNOWN/NOT SPECIFIED,BLACK/AFRICAN AMERICAN,PATIENT DECLINED TO ANSWER,OTHER,HISPANIC OR LATINO,HISPANIC/LATINO - GUATEMALAN,ASIAN - CHINESE,HISPANIC/LATINO - PUERTO RICAN,ASIAN - ASIAN INDIAN,ASIAN - VIETNAMESE,MULTI RACE ETHNICITY,HISPANIC/LATINO - DOMINICAN,AMERICAN INDIAN/ALASKA NATIVE,WHITE - RUSSIAN,BLACK/

In [79]:
df_pat.describe()

,SUBJECT_ID,LOS,HOSPITALIZATION,ORDINAL_AGE,ELECTIVE,EMERGENCY,NEWBORN,URGENT,Government,Medicaid,Medicare,Private,Self Pay,DIVORCED,LIFE PARTNER,MARRIED,NAN,SEPARATED,SINGLE,UNKNOWN,WIDOWED,GENDER,3,4,5,7,8,9,10,11,12,13,14,18,27,31,32,34,35,38,39,40,41,42,46,47,48,49,52,53,54,57,58,66,70,75,78,79,82,86,88,94,96,97,110,111,112,117,120,121,127,130,131,135,136,137,138,139,140,141,142,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,161,162,163,164,171,172,173,174,175,176,180,182,183,184,185,186,187,188,189,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,207,208,209,210,211,212,213,214,215,216,217,218,220,221,223,225,226,227,228,229,230,232,233,235,236,237,238,239,240,241,242,244,245,246,249,250,251,252,253,254,255,256,257,258,259,261,262,263,265,266,268,269,270,271,272,273,274,275,276,277,278,279,280,281,282,283,284,285,286,287,288,289,290,291,292,293,294,295,296,297,298,299,300,301,303,304,305,306,307,308,309,310,311,312,313,314,315,317,318,319,320,321,322,323,324,325,327,331,332,333,334,335,336,337,338,339,340,341,342,343,344,345,346,347,348,349,350,351,352,353,354,355,356,357,358,359,360,361,362,363,364,365,366,367,368,369,370,371,372,373,374,375,376,377,378,379,380,382,383,385,386,387,388,389,390,391,394,395,396,397,398,401,402,403,404,405,410,411,412,413,414,415,416,417,420,421,422,423,424,425,426,427,428,429,430,431,432,433,434,435,436,437,438,440,441,442,443,444,445,446,447,448,449,451,452,453,454,455,456,457,458,459,461,462,463,464,465,466,471,472,473,475,477,478,480,481,482,483,484,485,486,487,488,490,491,492,493,494,495,496,500,501,502,506,507,508,510,511,512,513,514,515,516,517,518,519,520,521,522,523,524,525,526,527,528,529,530,531,532,533,534,535,536,537,538,540,541,542,543,550,551,552,553,555,556,557,558,560,562,564,565,566,567,568,569,570,571,572,573,574,575,576,577,578,579,580,581,582,583,584,585,586,587,588,590,591,592,593,594,595,596,597,598,599,600,601,602,603,604,605,607,608,611,614,615,616,617,618,619,620,621,622,623,624,625,626,627,628,629,632,633,634,642,644,646,647,648,649,652,654,666,670,674,680,681,682,683,684,685,686,690,692,693,694,695,696,697,698,700,701,702,703,704,705,706,707,708,709,710,711,712,713,714,715,716,718,719,720,721,722,723,724,725,726,727,728,729,730,731,732,733,734,735,736,737,738,741,742,745,746,747,748,750,751,752,753,754,755,756,757,758,759,765,766,770,774,775,780,781,782,783,784,785,786,787,788,789,790,791,792,793,794,795,796,799,800,801,802,803,804,805,806,807,808,810,811,812,813,814,815,816,817,820,821,822,823,824,825,826,830,831,833,834,835,836,839,840,842,845,846,847,850,851,852,853,854,860,861,862,863,864,865,866,867,868,869,870,872,873,874,875,879,880,881,882,883,884,886,887,890,891,892,893,894,896,897,900,901,902,903,904,905,906,907,908,909,910,911,912,913,914,916,917,918,919,920,921,922,923,924,926,927,933,934,935,936,937,939,941,942,944,945,947,948,950,951,952,953,954,955,956,958,959,961,962,964,965,966,967,968,969,970,971,972,974,975,977,983,987,989,990,991,992,994,995,996,997,998,999,E812,E813,E814,E815,E816,E817,E818,E819,E821,E822,E823,E824,E825,E826,E838,E848,E849,E850,E851,E852,E853,E854,E855,E857,E858,E866,E869,E870,E871,E874,E876,E878,E879,E880,E881,E882,E884,E885,E887,E888,E890,E899,E900,E901,E906,E910,E911,E912,E915,E916,E917,E918,E920,E922,E924,E927,E928,E929,E930,E931,E932,E933,E934,E935,E936,E937,E938,E939,E940,E941,E942,E943,E944,E945,E946,E947,E950,E953,E955,E956,E957,E958,E960,E965,E966,E967,E968,E969,E980,E988,E989,V01,V02,V03,V04,V05,V06,V07,V08,V09,V10,V11,V12,V13,V14,V15,V16,V17,V18,V19,V23,V26,V27,V29,V30,V42,V43,V44,V45,V46,V49,V50,V53,V54,V55,V56,V58,V60,V61,V62,V63,V64,V65,V66,V69,V70,V72,V74,V76,DEAD,ASIAN,WHITE,UNKNOWN/NOT SPECIFIED,BLACK/AFRICAN AMERICAN,PATIENT DECLINED TO ANSWER,OTHER,HISPANIC OR LATINO,HISPANIC/LATINO - GUATEMALAN,ASIAN - CHINESE,HISPANIC/LATINO - PUERTO RICAN,ASIAN - ASIAN INDIAN,ASIAN - VIETNAMESE,MULTI RACE ETHNICITY,HISPANIC/LATINO - DOMINICAN,AMERICAN INDIAN/ALASKA NATIVE,WHITE - RUSSIAN,BLACK/

In [104]:

print('\n Cluster' + str(1+1) + '\n Summary Statistics \n')
pat_clustered[1].describe()


 Cluster2
 Summary Statistics 



,SUBJECT_ID,LOS,HOSPITALIZATION,ORDINAL_AGE,ELECTIVE,EMERGENCY,NEWBORN,URGENT,Government,Medicaid,Medicare,Private,Self Pay,DIVORCED,LIFE PARTNER,MARRIED,NAN,SEPARATED,SINGLE,UNKNOWN,WIDOWED,GENDER,3,4,5,7,8,9,10,11,12,13,14,18,27,31,32,34,35,38,39,40,41,42,46,47,48,49,52,53,54,57,58,66,70,75,78,79,82,86,88,94,96,97,110,111,112,117,120,121,127,130,131,135,136,137,138,139,140,141,142,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,161,162,163,164,171,172,173,174,175,176,180,182,183,184,185,186,187,188,189,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,207,208,209,210,211,212,213,214,215,216,217,218,220,221,223,225,226,227,228,229,230,232,233,235,236,237,238,239,240,241,242,244,245,246,249,250,251,252,253,254,255,256,257,258,259,261,262,263,265,266,268,269,270,271,272,273,274,275,276,277,278,279,280,281,282,283,284,285,286,287,288,289,290,291,292,293,294,295,296,297,298,299,300,301,303,304,305,306,307,308,309,310,311,312,313,314,315,317,318,319,320,321,322,323,324,325,327,331,332,333,334,335,336,337,338,339,340,341,342,343,344,345,346,347,348,349,350,351,352,353,354,355,356,357,358,359,360,361,362,363,364,365,366,367,368,369,370,371,372,373,374,375,376,377,378,379,380,382,383,385,386,387,388,389,390,391,394,395,396,397,398,401,402,403,404,405,410,411,412,413,414,415,416,417,420,421,422,423,424,425,426,427,428,429,430,431,432,433,434,435,436,437,438,440,441,442,443,444,445,446,447,448,449,451,452,453,454,455,456,457,458,459,461,462,463,464,465,466,471,472,473,475,477,478,480,481,482,483,484,485,486,487,488,490,491,492,493,494,495,496,500,501,502,506,507,508,510,511,512,513,514,515,516,517,518,519,520,521,522,523,524,525,526,527,528,529,530,531,532,533,534,535,536,537,538,540,541,542,543,550,551,552,553,555,556,557,558,560,562,564,565,566,567,568,569,570,571,572,573,574,575,576,577,578,579,580,581,582,583,584,585,586,587,588,590,591,592,593,594,595,596,597,598,599,600,601,602,603,604,605,607,608,611,614,615,616,617,618,619,620,621,622,623,624,625,626,627,628,629,632,633,634,642,644,646,647,648,649,652,654,666,670,674,680,681,682,683,684,685,686,690,692,693,694,695,696,697,698,700,701,702,703,704,705,706,707,708,709,710,711,712,713,714,715,716,718,719,720,721,722,723,724,725,726,727,728,729,730,731,732,733,734,735,736,737,738,741,742,745,746,747,748,750,751,752,753,754,755,756,757,758,759,765,766,770,774,775,780,781,782,783,784,785,786,787,788,789,790,791,792,793,794,795,796,799,800,801,802,803,804,805,806,807,808,810,811,812,813,814,815,816,817,820,821,822,823,824,825,826,830,831,833,834,835,836,839,840,842,845,846,847,850,851,852,853,854,860,861,862,863,864,865,866,867,868,869,870,872,873,874,875,879,880,881,882,883,884,886,887,890,891,892,893,894,896,897,900,901,902,903,904,905,906,907,908,909,910,911,912,913,914,916,917,918,919,920,921,922,923,924,926,927,933,934,935,936,937,939,941,942,944,945,947,948,950,951,952,953,954,955,956,958,959,961,962,964,965,966,967,968,969,970,971,972,974,975,977,983,987,989,990,991,992,994,995,996,997,998,999,E812,E813,E814,E815,E816,E817,E818,E819,E821,E822,E823,E824,E825,E826,E838,E848,E849,E850,E851,E852,E853,E854,E855,E857,E858,E866,E869,E870,E871,E874,E876,E878,E879,E880,E881,E882,E884,E885,E887,E888,E890,E899,E900,E901,E906,E910,E911,E912,E915,E916,E917,E918,E920,E922,E924,E927,E928,E929,E930,E931,E932,E933,E934,E935,E936,E937,E938,E939,E940,E941,E942,E943,E944,E945,E946,E947,E950,E953,E955,E956,E957,E958,E960,E965,E966,E967,E968,E969,E980,E988,E989,V01,V02,V03,V04,V05,V06,V07,V08,V09,V10,V11,V12,V13,V14,V15,V16,V17,V18,V19,V23,V26,V27,V29,V30,V42,V43,V44,V45,V46,V49,V50,V53,V54,V55,V56,V58,V60,V61,V62,V63,V64,V65,V66,V69,V70,V72,V74,V76,DEAD,ASIAN,WHITE,UNKNOWN/NOT SPECIFIED,BLACK/AFRICAN AMERICAN,PATIENT DECLINED TO ANSWER,OTHER,HISPANIC OR LATINO,HISPANIC/LATINO - GUATEMALAN,ASIAN - CHINESE,HISPANIC/LATINO - PUERTO RICAN,ASIAN - ASIAN INDIAN,ASIAN - VIETNAMESE,MULTI RACE ETHNICITY,HISPANIC/LATINO - DOMINICAN,AMERICAN INDIAN/ALASKA NATIVE,WHITE - RUSSIAN,BLACK/

In [75]:
for i in range(5):
    print(df_pat.columns[i])


SUBJECT_ID
LOS
HOSPITALIZATION
ORDINAL_AGE
ELECTIVE


In [76]:
for i in range(5):
    #print (pat_clustered[i])
    print('\n Cluster' + str(i+1) + 'Summary Statistics \n')
    a = df_pat.iloc[:, i]
    print(a.describe())


 Cluster1Summary Statistics 

count    10310.000000
mean     38591.982735
std      29268.731006
min         13.000000
25%      14379.250000
50%      28291.500000
75%      63174.500000
max      99999.000000
Name: SUBJECT_ID, dtype: float64

 Cluster2Summary Statistics 

count    10310.000000
mean         6.702829
std         10.604351
min          0.000600
25%          1.537200
50%          3.085100
75%          7.024100
max        207.130600
Name: LOS, dtype: float64

 Cluster3Summary Statistics 

count    10310.000000
mean         1.506208
std          1.347733
min          1.000000
25%          1.000000
50%          1.000000
75%          2.000000
max         42.000000
Name: HOSPITALIZATION, dtype: float64

 Cluster4Summary Statistics 

count    10310.000000
mean         7.278855
std          1.452093
min          1.000000
25%          6.000000
50%          7.000000
75%          8.000000
max         10.000000
Name: ORDINAL_AGE, dtype: float64

 Cluster5Summary Statistics 

count    1

In [101]:
printcluster(hinge4, df_pat, 4)


 Cluster 1: 

Linear Regression: 
(1947, 919) (1947,)
(650, 919) (650,)
Score: 6.732809352132833e+21 

Decission Tree: 
(1947, 919) (1947,)
(650, 919) (650,)
Score: 78.79608559686153 


 Cluster 2: 

Linear Regression: 
(1937, 919) (1937,)
(646, 919) (646,)
Score: 3.110152984790911e+23 

Decission Tree: 
(1937, 919) (1937,)
(646, 919) (646,)
Score: 101.3503159973839 


 Cluster 3: 

Linear Regression: 
(2028, 919) (2028,)
(676, 919) (676,)
Score: 5.158283577184971e+23 

Decission Tree: 
(2028, 919) (2028,)
(676, 919) (676,)
Score: 112.26858978699705 


 Cluster 4: 

Linear Regression: 
(1819, 919) (1819,)
(607, 919) (607,)
Score: 2.2250058585256225e+23 

Decission Tree: 
(1819, 919) (1819,)
(607, 919) (607,)
Score: 137.0759573405354 


 Cluster1Summary Statistics 


 Cluster2Summary Statistics 


 Cluster3Summary Statistics 


 Cluster4Summary Statistics 



In [56]:
printcluster(param100_4, df_pat, 4)


 Cluster 1: 

Linear Regression: 
(1811, 919) (1811,)
(604, 919) (604,)
Score: 1.7139008308893867e+24 

Decission Tree: 
(1811, 919) (1811,)
(604, 919) (604,)
Score: 168.50718526268625 


 Cluster 2: 

Linear Regression: 
(2017, 919) (2017,)
(673, 919) (673,)
Score: 1.216336134316896e+22 

Decission Tree: 
(2017, 919) (2017,)
(673, 919) (673,)
Score: 106.02576964619986 


 Cluster 3: 

Linear Regression: 
(1970, 919) (1970,)
(657, 919) (657,)
Score: 1.3019256201777722e+17 

Decission Tree: 
(1970, 919) (1970,)
(657, 919) (657,)
Score: 75.20957387498098 


 Cluster 4: 

Linear Regression: 
(1933, 919) (1933,)
(645, 919) (645,)
Score: 6.291130141474561e+23 

Decission Tree: 
(1933, 919) (1933,)
(645, 919) (645,)
Score: 122.49535126706976 


 Cluster1Summary Statistics
         SUBJECT_ID          LOS  HOSPITALIZATION  ORDINAL_AGE     ELECTIVE    EMERGENCY  NEWBORN       URGENT   Government     Medicaid     Medicare      Private     Self Pay     DIVORCED  LIFE PARTNER      MARRIED       

         SUBJECT_ID          LOS  HOSPITALIZATION  ORDINAL_AGE     ELECTIVE    EMERGENCY  NEWBORN       URGENT   Government     Medicaid     Medicare      Private     Self Pay     DIVORCED  LIFE PARTNER      MARRIED          NAN    SEPARATED       SINGLE      UNKNOWN      WIDOWED       GENDER       3            4       5       7            8            9      10           11      12      13      14      18      27      31           32      34           35           38      39           40           41           42      46           47      48           49           52           53           54           57           58      66           70      75           78          79      82      86           88      94      96      97          110          111          112          117     120     121          127     130          131         135          136     137          138     139          140     141     142          144          145     146     147          148          149  \
count   26

         SUBJECT_ID          LOS  HOSPITALIZATION  ORDINAL_AGE     ELECTIVE    EMERGENCY      NEWBORN       URGENT   Government     Medicaid     Medicare      Private     Self Pay     DIVORCED  LIFE PARTNER      MARRIED          NAN    SEPARATED       SINGLE      UNKNOWN      WIDOWED       GENDER       3       4            5       7            8            9      10           11      12      13      14      18      27           31           32      34           35           38      39           40           41           42      46           47           48      49           52           53           54      57      58      66           70      75           78           79      82           86           88           94      96           97          110          111          112          117     120          121     127          130     131          135          136          137          138     139     140     141     142     144     145     146     147          148     149  \
count   2

         SUBJECT_ID          LOS  HOSPITALIZATION  ORDINAL_AGE     ELECTIVE    EMERGENCY  NEWBORN       URGENT   Government     Medicaid     Medicare      Private     Self Pay     DIVORCED  LIFE PARTNER      MARRIED          NAN    SEPARATED      SINGLE      UNKNOWN      WIDOWED       GENDER            3       4       5       7            8            9      10           11      12           13           14      18      27           31      32           34      35           38           39           40           41           42           46           47      48           49      52           53           54      57           58           66           70      75           78           79           82      86           88           94      96      97          110     111          112          117     120     121          127     130          131          135          136     137          138          139          140     141          142     144          145          146     147  \
count

In [37]:
printcluster(param7, df_pat, 7)


 Cluster 1: 

Linear Regression: 
(1092, 919) (1092,)
(365, 919) (365,)
Score: 1.4633745151954942e+23 

Decission Tree: 
(1092, 919) (1092,)
(365, 919) (365,)
Score: 68.29840452511645 


 Cluster 2: 

Linear Regression: 
(1136, 919) (1136,)
(379, 919) (379,)
Score: 5.591221584103141e+22 

Decission Tree: 
(1136, 919) (1136,)
(379, 919) (379,)
Score: 174.87557868422167 


 Cluster 3: 

Linear Regression: 
(1071, 919) (1071,)
(357, 919) (357,)
Score: 1.9008154531598883e+22 

Decission Tree: 
(1071, 919) (1071,)
(357, 919) (357,)
Score: 144.2299990938095 


 Cluster 4: 

Linear Regression: 
(1095, 919) (1095,)
(366, 919) (366,)
Score: 7.808771755970933e+23 

Decission Tree: 
(1095, 919) (1095,)
(366, 919) (366,)
Score: 143.6242499447541 


 Cluster 5: 

Linear Regression: 
(1131, 919) (1131,)
(378, 919) (378,)
Score: 6.131651108091408e+21 

Decission Tree: 
(1131, 919) (1131,)
(378, 919) (378,)
Score: 136.40025158584655 


 Cluster 6: 

Linear Regression: 
(1097, 919) (1097,)
(366, 919) 

         SUBJECT_ID          LOS  HOSPITALIZATION  ORDINAL_AGE     ELECTIVE  \
count   1428.000000  1428.000000      1428.000000  1428.000000  1428.000000   
mean   39331.769608     6.421495         1.488095     7.315826     0.203782   
std    29669.409260    10.374594         1.075561     1.464497     0.402950   
min       55.000000     0.043100         1.000000     2.000000     0.000000   
25%    14768.500000     1.493175         1.000000     6.000000     0.000000   
50%    28932.500000     3.116500         1.000000     7.000000     0.000000   
75%    66505.000000     7.082025         2.000000     8.000000     0.000000   
max    99802.000000   207.130600        12.000000    10.000000     1.000000   

         EMERGENCY  NEWBORN       URGENT   Government     Medicaid  ...  \
count  1428.000000   1428.0  1428.000000  1428.000000  1428.000000  ...   
mean      0.840336      0.0     0.042717     0.020308     0.082633  ...   
std       0.366422      0.0     0.202289     0.141102     0.275

         SUBJECT_ID          LOS  HOSPITALIZATION  ORDINAL_AGE     ELECTIVE  \
count   1509.000000  1509.000000      1509.000000  1509.000000  1509.000000   
mean   37184.070245     6.780592         1.524851     7.220013     0.183565   
std    28951.501892    11.000581         1.589571     1.442293     0.387258   
min       35.000000     0.039400         1.000000     2.000000     0.000000   
25%    13145.000000     1.595100         1.000000     6.000000     0.000000   
50%    27935.000000     3.124300         1.000000     7.000000     0.000000   
75%    59728.000000     7.056200         2.000000     8.000000     0.000000   
max    99938.000000   109.723600        34.000000    10.000000     1.000000   

         EMERGENCY  NEWBORN       URGENT   Government     Medicaid  ...  \
count  1509.000000   1509.0  1509.000000  1509.000000  1509.000000  ...   
mean      0.846256      0.0     0.036448     0.025182     0.074884  ...   
std       0.360823      0.0     0.187464     0.156730     0.263

         SUBJECT_ID          LOS  HOSPITALIZATION  ORDINAL_AGE     ELECTIVE  \
count   1477.000000  1477.000000      1477.000000  1477.000000  1477.000000   
mean   38086.287745     6.502853         1.496953     7.251862     0.181449   
std    29215.841293     9.780447         1.370725     1.441804     0.385520   
min      186.000000     0.016400         1.000000     2.000000     0.000000   
25%    14394.000000     1.547700         1.000000     6.000000     0.000000   
50%    28062.000000     3.088800         1.000000     7.000000     0.000000   
75%    62603.000000     6.746000         2.000000     8.000000     0.000000   
max    99901.000000   123.024200        24.000000    10.000000     1.000000   

         EMERGENCY  NEWBORN       URGENT   Government     Medicaid  ...  \
count  1477.000000   1477.0  1477.000000  1477.000000  1477.000000  ...   
mean      0.846987      0.0     0.042654     0.028436     0.087339  ...   
std       0.360122      0.0     0.202144     0.166271     0.282

In [26]:
printcluster(hinge7, df_pat, 7)

       SUBJECT_ID      LOS  HOSPITALIZATION  ORDINAL_AGE  ELECTIVE  EMERGENCY  \
0              13   3.6660                1            4         0          1   
11             75   1.2469                1            8         1          0   
30            224   1.1046                1            8         0          1   
39            271   2.0600                1            5         0          1   
40            286  56.6779                2            9         0          1   
...           ...      ...              ...          ...       ...        ...   
10258       99356   3.2067                1            8         0          1   
10260       99361  10.9524                2            8         0          1   
10268       99467   1.6653                1            9         0          1   
10284       99756   2.5346                3            6         0          1   
10305       99955   5.6719                1            7         0          1   

       NEWBORN  URGENT  Gov

       SUBJECT_ID      LOS  HOSPITALIZATION  ORDINAL_AGE  ELECTIVE  EMERGENCY  \
4              24   0.5124                1            4         0          1   
15            130  17.4592                2            7         0          1   
19            157   6.1813                2            9         0          1   
20            173   1.3722                1            7         1          0   
27            205   2.9076                1            8         0          0   
...           ...      ...              ...          ...       ...        ...   
10240       99085   8.2287                4            5         1          1   
10269       99469  29.6298                3            7         0          1   
10270       99485   2.2192                1            7         0          1   
10271       99487   0.8865                1            7         0          0   
10274       99538  12.0662                2            8         0          1   

       NEWBORN  URGENT  Gov

Score: 59.47073430018616 

       SUBJECT_ID      LOS  HOSPITALIZATION  ORDINAL_AGE  ELECTIVE  EMERGENCY  \
14            123   1.4941                1            6         0          1   
21            184   5.0941                2            6         0          1   
22            186   3.7078                1            8         0          1   
47            342   2.2438                1            8         0          1   
54            357  94.8967                5            7         0          1   
...           ...      ...              ...          ...       ...        ...   
10283       99714   4.0187                1            9         0          1   
10294       99863  28.7306                1            5         0          1   
10297       99893   2.1240                1            9         0          1   
10301       99903   1.0388                1            6         0          1   
10309       99999   1.1242                1            7         1          0   



Score: 155.83936969097257 

       SUBJECT_ID      LOS  HOSPITALIZATION  ORDINAL_AGE  ELECTIVE  EMERGENCY  \
3              21  14.2664                2            9         0          1   
12             83   2.3141                1            7         0          0   
32            235   0.0394                1            8         0          1   
43            309   1.3873                1            7         0          1   
55            359   3.3826                1            5         0          1   
...           ...      ...              ...          ...       ...        ...   
10272       99491  27.2813                1            9         0          1   
10273       99514   0.9357                1            8         0          1   
10288       99806   2.6988                1            7         0          1   
10290       99810   1.6393                1            6         1          0   
10308       99995   2.1615                1            9         1          0   


In [38]:
printcluster(param100_7, df_pat, 7)


 Cluster 1: 

Linear Regression: 
(1275, 919) (1275,)
(426, 919) (426,)
Score: 3.329769683358834e+20 

Decission Tree: 
(1275, 919) (1275,)
(426, 919) (426,)
Score: 80.7615908579108 


 Cluster 2: 

Linear Regression: 
(1092, 919) (1092,)
(365, 919) (365,)
Score: 2.212449528124347e+22 

Decission Tree: 
(1092, 919) (1092,)
(365, 919) (365,)
Score: 137.94068864241095 


 Cluster 3: 

Linear Regression: 
(1026, 919) (1026,)
(342, 919) (342,)
Score: 2.052131746386652e+20 

Decission Tree: 
(1026, 919) (1026,)
(342, 919) (342,)
Score: 129.72150378722222 


 Cluster 4: 

Linear Regression: 
(1064, 919) (1064,)
(355, 919) (355,)
Score: 4.692254253047854e+22 

Decission Tree: 
(1064, 919) (1064,)
(355, 919) (355,)
Score: 148.46466316152816 


 Cluster 5: 

Linear Regression: 
(1155, 919) (1155,)
(386, 919) (386,)
Score: 9.033468977479742e+23 

Decission Tree: 
(1155, 919) (1155,)
(386, 919) (386,)
Score: 127.59352055241581 


 Cluster 6: 

Linear Regression: 
(1101, 919) (1101,)
(368, 919) (

         SUBJECT_ID          LOS  HOSPITALIZATION  ORDINAL_AGE     ELECTIVE  \
count   1368.000000  1368.000000      1368.000000  1368.000000  1368.000000   
mean   38709.260234     6.855026         1.546053     7.247076     0.163012   
std    29489.587355    10.384985         1.470286     1.450094     0.369511   
min       35.000000     0.056600         1.000000     2.000000     0.000000   
25%    13905.500000     1.546100         1.000000     6.000000     0.000000   
50%    28591.000000     3.143250         1.000000     7.000000     0.000000   
75%    65438.000000     7.331975         2.000000     8.000000     0.000000   
max    99928.000000   128.960500        23.000000    10.000000     1.000000   

         EMERGENCY  NEWBORN       URGENT   Government     Medicaid  ...  \
count  1368.000000   1368.0  1368.000000  1368.000000  1368.000000  ...   
mean      0.877193      0.0     0.038012     0.023392     0.089181  ...   
std       0.328336      0.0     0.191294     0.151200     0.285

         SUBJECT_ID          LOS  HOSPITALIZATION  ORDINAL_AGE     ELECTIVE  \
count   1541.000000  1541.000000      1541.000000  1541.000000  1541.000000   
mean   38009.439325     6.667941         1.518494     7.299156     0.179104   
std    29266.223535    10.894417         1.263042     1.469846     0.383564   
min       25.000000     0.121900         1.000000     1.000000     0.000000   
25%    14042.000000     1.520600         1.000000     6.000000     0.000000   
50%    27892.000000     2.964900         1.000000     7.000000     0.000000   
75%    61030.000000     6.746000         2.000000     8.000000     0.000000   
max    99938.000000   152.731800        20.000000    10.000000     1.000000   

         EMERGENCY      NEWBORN       URGENT   Government     Medicaid  ...  \
count  1541.000000  1541.000000  1541.000000  1541.000000  1541.000000  ...   
mean      0.855938     0.000649     0.038936     0.026606     0.092148  ...   
std       0.351267     0.025474     0.193505     0.

         SUBJECT_ID          LOS  HOSPITALIZATION  ORDINAL_AGE     ELECTIVE  \
count   1355.000000  1355.000000      1355.000000  1355.000000  1355.000000   
mean   39210.185978     6.578088         1.541697     7.222140     0.198524   
std    29361.396517    10.221371         1.375668     1.474083     0.399036   
min       21.000000     0.000600         1.000000     2.000000     0.000000   
25%    14898.500000     1.477700         1.000000     6.000000     0.000000   
50%    29010.000000     3.046500         1.000000     7.000000     0.000000   
75%    63661.000000     6.687100         2.000000     8.000000     0.000000   
max    99814.000000   118.816700        24.000000    10.000000     1.000000   

         EMERGENCY  NEWBORN       URGENT   Government     Medicaid  ...  \
count  1355.000000   1355.0  1355.000000  1355.000000  1355.000000  ...   
mean      0.847232      0.0     0.043542     0.022878     0.075277  ...   
std       0.359896      0.0     0.204150     0.149570     0.263

In [39]:
printcluster(param100_6, df_pat, 6)


 Cluster 1: 

Linear Regression: 
(1218, 919) (1218,)
(406, 919) (406,)
Score: 3.254509775602841e+23 

Decission Tree: 
(1218, 919) (1218,)
(406, 919) (406,)
Score: 151.78809629320196 


 Cluster 2: 

Linear Regression: 
(1322, 919) (1322,)
(441, 919) (441,)
Score: 1.0106048451563662e+21 

Decission Tree: 
(1322, 919) (1322,)
(441, 919) (441,)
Score: 95.50143088963719 


 Cluster 3: 

Linear Regression: 
(1268, 919) (1268,)
(423, 919) (423,)
Score: 4.601591772741882e+21 

Decission Tree: 
(1268, 919) (1268,)
(423, 919) (423,)
Score: 133.25151912224587 


 Cluster 4: 

Linear Regression: 
(1341, 919) (1341,)
(447, 919) (447,)
Score: 1.1343626919828798e+23 

Decission Tree: 
(1341, 919) (1341,)
(447, 919) (447,)
Score: 148.86979997022368 


 Cluster 5: 

Linear Regression: 
(1268, 919) (1268,)
(423, 919) (423,)
Score: 1.577132796339587e+23 

Decission Tree: 
(1268, 919) (1268,)
(423, 919) (423,)
Score: 69.70207844057327 


 Cluster 6: 

Linear Regression: 
(1314, 919) (1314,)
(439, 919)

         SUBJECT_ID          LOS  HOSPITALIZATION  ORDINAL_AGE     ELECTIVE  \
count   1691.000000  1691.000000      1691.000000  1691.000000  1691.000000   
mean   38496.937315     7.154120         1.534595     7.337670     0.174453   
std    29064.187044    11.282501         1.425616     1.467234     0.379611   
min       24.000000     0.071300         1.000000     2.000000     0.000000   
25%    14510.000000     1.663850         1.000000     6.000000     0.000000   
50%    29168.000000     3.186800         1.000000     7.000000     0.000000   
75%    61449.500000     7.249200         2.000000     8.000000     0.000000   
max    99903.000000   152.731800        24.000000    10.000000     1.000000   

         EMERGENCY  NEWBORN       URGENT   Government     Medicaid  ...  \
count  1691.000000   1691.0  1691.000000  1691.000000  1691.000000  ...   
mean      0.860438      0.0     0.034299     0.021289     0.078652  ...   
std       0.346635      0.0     0.182051     0.144389     0.269

         SUBJECT_ID          LOS  HOSPITALIZATION  ORDINAL_AGE     ELECTIVE  \
count   1691.000000  1691.000000      1691.000000  1691.000000  1691.000000   
mean   40331.133649     6.338280         1.470727     7.284447     0.181549   
std    29761.908638     9.713744         1.166926     1.419263     0.385587   
min       13.000000     0.043100         1.000000     1.000000     0.000000   
25%    15776.000000     1.520950         1.000000     6.000000     0.000000   
50%    29231.000000     3.026200         1.000000     7.000000     0.000000   
75%    66021.000000     6.445300         2.000000     8.000000     0.000000   
max    99999.000000   123.024200        20.000000    10.000000     1.000000   

         EMERGENCY      NEWBORN       URGENT   Government     Medicaid  ...  \
count  1691.000000  1691.000000  1691.000000  1691.000000  1691.000000  ...   
mean      0.856298     0.000591     0.041396     0.024837     0.079243  ...   
std       0.350891     0.024318     0.199262     0.

In [ ]:
pr